In [1]:
from datasets import load_dataset
import nltk
nltk.download('punkt')

# Model from HugginFace Account
model_name = "google-t5/t5-small"

# Dataset to augment/train/finetune model for summarization
dataset_name = "kmfoda/booksum"

# Load the dataset
dataset = load_dataset(dataset_name)


[nltk_data] Downloading package punkt to
[nltk_data]     /home/emmanuelmoran/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


#### Dont forget to put you HFH API KEY

In [2]:
from huggingface_hub import HfApi, HfFolder
api         = HfApi()
api_token   = 'INPUT HUGGING FACE API KEY'
HfFolder.save_token(api_token)
user        = api.whoami()
print(f"Logged in as: {user['name']}")

Logged in as: fnando1995


In [3]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq

# Load tokenizers, model and the data collator
tokenizer       = AutoTokenizer.from_pretrained(model_name)
model           = AutoModelForSeq2SeqLM.from_pretrained(model_name)
data_collator   = DataCollatorForSeq2Seq(tokenizer, model=model)

In [4]:
# Function to preprocess dataset examples
def preprocess_data(examples):
    inputs = [ f'summarize the following chapter: {example}\nsummary:' for example in examples['chapter']]
    targets = examples['summary']
    
    model_inputs    = tokenizer(inputs, max_length=512, truncation=True, padding="max_length")
    labels          = tokenizer(targets, max_length=150, truncation=True, padding="max_length")
    
    model_inputs["labels"] = labels["input_ids"]
    
    return model_inputs

# Apply preprocessing
tokenized_datasets = dataset.map(preprocess_data, batched=True)


Map:   0%|          | 0/1484 [00:00<?, ? examples/s]

In [5]:
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer

training_args = Seq2SeqTrainingArguments(
    f"{model_name}-ft-bookSum",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=10,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=True,
)


/media/emmanuelmoran/Emmanuel/lumenalta/llm-assessment/llm-assessment/env/lib/python3.10/site-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [6]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

In [7]:
trainer.train()

  0%|          | 0/24000 [00:00<?, ?it/s]

KeyboardInterrupt: 